<a href="https://colab.research.google.com/github/shu-admin/NeuralNetwork-notebook/blob/main/NeuralNetwork_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip3 install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1.9MB 2.8MB/s 


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

python中的super(Net, self).__init__()
 
首先找到Net的父类（比如是类NNet），然后把类Net的对象self转换为类NNet的对象，然后“被转换”的类NNet对象调用自己的init函数

In [10]:
class Net(nn.Module):
  
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1,6,5)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1  = nn.Linear(16*5*5,120)
    self.fc2  = nn.Linear(120,84)
    self.fc3  = nn.Linear(84,10)

  def forward(self,x):
    x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)),2)
    x = x.view(-1,self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self,x):
    size = x.size()[1:]
    num_features = 1
    for s in size:
      num_features *=s
    return num_features
  



In [11]:
net = Net()
print(next)

<built-in function next>


In [8]:
torch.randn(1,1,32,32)

tensor([[[[ 1.8178,  0.6415, -0.4277,  ..., -0.5380,  0.0741, -0.8095],
          [ 0.4341,  0.8842,  0.7778,  ...,  0.6204,  1.2489, -1.1258],
          [-0.5250,  1.8733, -0.6832,  ..., -0.2369, -1.2173, -0.3649],
          ...,
          [-0.7369, -0.9011,  1.3844,  ..., -1.1841,  0.4685, -0.8936],
          [ 1.0979,  0.6499,  0.3448,  ..., -1.2502, -0.1374,  0.3688],
          [ 0.2469, -0.2894,  1.3907,  ...,  1.0479,  0.8580,  0.0217]]]])

In [12]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0779, -0.0375,  0.1755,  0.0235, -0.0017, -0.0762, -0.0285,  0.0404,
         -0.1282,  0.0478]], grad_fn=<AddmmBackward>)


清零所有参数的梯度缓存，进行随机梯度的反向传播

In [13]:
net.zero_grad()#清零梯度缓存
out.backward(torch.randn(1,10))

**LOSS损失函数**

一个损失函数接受一对(output, target)作为输入，计算一个值来估计网络的输出和目标值相差多少
nn.MSELoss计算输出和目标的均方误差(mean-squared error)

In [22]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(output,target)
print(loss)

tensor(1.3156, grad_fn=<MseLossBackward>)


In [17]:
print(loss.grad_fn)

反向传播误差

In [28]:
net.zero_grad()#清除参数的梯度
print("未反向传播的梯度：",net.conv1.bias.grad)
loss.backward(retain_graph=True)
print("反向传播之后的梯度：",net.conv1.bias.grad)

未反向传播的梯度： tensor([0., 0., 0., 0., 0., 0.])
反向传播之后的梯度： tensor([ 0.0033,  0.0142,  0.0035,  0.0095, -0.0212, -0.0204])


**更新权重**

SGD随机梯度下降法
>SGD算法效率高，每次随机选择一个mini-batch去计算梯度，在minibatch-loss上的梯度显然是original-loss上的梯度的无偏估计，因此利用minibatch-loss上的梯度可以近似original-loss上的梯度，并且每走一步只需要遍历一个minibatch（一～几百）的数据。<br>
<br>
> weight = weight - learning_rate * gradient

​	

In [31]:
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(learning_rate * f.grad.data)
  print(f.data)
#官方给了一种优化
#可以使用torch.optim
#optimizer = optim.SGD(net.parameters(),learning_rate)

tensor([[[[ 0.0690, -0.1547,  0.1085,  0.0669, -0.0847],
          [ 0.1635, -0.1473,  0.0665, -0.1130,  0.0633],
          [-0.0602, -0.1616,  0.0756,  0.1222, -0.1256],
          [-0.1570,  0.1239, -0.1683,  0.1423, -0.1586],
          [-0.0468, -0.1204, -0.0509, -0.1909,  0.1294]]],


        [[[ 0.0424, -0.1151,  0.1043,  0.0119, -0.0924],
          [-0.0835, -0.1650, -0.0679,  0.0566, -0.0468],
          [-0.0228, -0.0015, -0.0696, -0.0216,  0.0949],
          [-0.0945,  0.1118, -0.0824,  0.1842,  0.1202],
          [-0.1431, -0.0126,  0.0170, -0.0194, -0.0850]]],


        [[[ 0.0490,  0.0690,  0.0914,  0.0444,  0.0178],
          [ 0.1851,  0.0240, -0.0002,  0.1503, -0.1970],
          [-0.0971, -0.0508,  0.0101,  0.0130, -0.1099],
          [-0.1921, -0.0730,  0.0573,  0.0568,  0.0445],
          [-0.1939, -0.0385, -0.1551, -0.1298,  0.1839]]],


        [[[ 0.0118, -0.0259, -0.1413,  0.0980, -0.1356],
          [-0.0561,  0.1614, -0.1885, -0.0025,  0.0007],
          [ 0.1833,

In [35]:
#完整的流程
import torch.optim as optim
optimizer = optim.SGD(net.parameters(),learning_rate)

optimizer.zero_grad()
#清除梯度缓存
output = net(input)
loss = criterion(output,target)#计算损失函数
loss.backward()
optimizer.step()
#更新参数